In [6]:
import pandas as pd
import numpy as np
import requests
import re
import praw
import nltk
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
# Initialize Reddit instance (ensure you have your Reddit API credentials)
reddit = praw.Reddit(
    client_id='c0ZCRR0k3I4y-5u0R_XtLg',
    client_secret='nH2svexjVfPAZOkxJ8QsCqF_1SOOdg',
    user_agent='data science final project'
)


In [ ]:
# Choose subreddits related to cryptocurrencies
subreddits = ['Bitcoin', 'CryptoCurrency', 'btc', 'cryptomarkets']

def fetch_reddit_posts(subreddit_list, limit=2000):
    posts = []
    for subreddit in subreddit_list:
        try:
            subreddit_obj = reddit.subreddit(subreddit)
            for post in subreddit_obj.new(limit=limit):
                posts.append({
                    'timestamp': post.created_utc,
                    'title': post.title,
                    'body': post.selftext,
                    'score': post.score,
                    'num_comments': post.num_comments,
                    'author': str(post.author),
                    'subreddit': subreddit
                })
        except Exception as e:
            print(f"An error occurred while fetching posts from subreddit {subreddit}: {e}")
    df = pd.DataFrame(posts)
    return df

reddit_posts = fetch_reddit_posts(subreddits)



In [15]:
reddit_posts


,timestamp,title,body,score,num_comments,author,subreddit
0,1.731647e+09,Jay Clayton,Is Jay Clayton going to hurt bitcoin?\n\nHis a...,2,1,BitcoinLibertarian,Bitcoin
1,1.731646e+09,Newb have a lot of Fiat to spend finally wakin...,I believe in this I just want to maximize what...,0,3,Lanks803,Bitcoin
2,1.731645e+09,Why Bitcoin is LOVE-GO-UP Technology - A Podca...,,2,1,BENshakalaka,Bitcoin
3,1.731642e+09,How can I actually own a bitcoin?,I've had an autobuy setup in Robin hood for so...,3,12,Cupcake_Shake,Bitcoin
4,1.731642e+09,Pennsylvania House Moves To Add Bitcoin To Sta...,And so it begins,94,4,benitoblanco888,Bitcoin
...,...,...,...,...,...,...,...
3257,1.727275e+09,"Orderly Network, Google Cloud expand AI bounty...",,31,7,absurdcriminality,cryptomarkets
3258,1.727271e+09,Binance And ChainGPT Partner On Major Campaign,,0,1,gamefidelio,cryptomarkets
3259,1.727268e+09,Divergence Trading-Legit Setup or Just Noise?,"So, I have read quite a lot about divergence i...",0,1,Spunky-Sprout,cryptomarkets
3260,1.727267e+09,Martingale Strategy and Leverage: A Dangerous ...,I've heard some people use the Martingale stra...,2,4,Snore_Face_Siren,cryptomarkets


In [17]:
def get_price_data(symbol='BTCUSDT', interval='1h', limit=1000):
    url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(url, params=params, timeout=10)
    data = response.json()
    # Extract relevant fields
    df = pd.DataFrame(data, columns=[
        'Open_Time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close_Time', 'Quote_Asset_Volume', 'Number_of_Trades',
        'Taker_Buy_Base_Asset_Volume', 'Taker_Buy_Quote_Asset_Volume', 'Ignore'
    ])
    df['Open_Time'] = pd.to_datetime(df['Open_Time'], unit='ms')
    df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit='ms')
    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    return df

price_data = get_price_data()

ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)